In [ ]:
import matplotlib
matplotlib.use("PDF")
from pylab import *
import sys, glob, xlrd, datetime
from scipy import stats
thedir='/Users/cslage/Research/LSST/optical_simulator/timing/ltspice/sta3800_output_device'
%cd $thedir


In [ ]:
data_wb = xlrd.open_workbook('STA3800_meas.xls')
idvd1_data = data_wb.sheet_by_name('8-1010.TXT')
idvd2_data = data_wb.sheet_by_name('8-1015.TXT')
idvg1_data = data_wb.sheet_by_name('8-1011.TXT')
idvg2_data = data_wb.sheet_by_name('8-1013.TXT')
idvg3_data = data_wb.sheet_by_name('8-1014.TXT')
cv_data = data_wb.sheet_by_name('C-V')


In [ ]:
# ID-VG 1
W= 28.0
L = 5.0
Mu = 1000.0
Vbs = 0.0
Vds = 0.50
Vgs = []
Ids = []
for i in range(idvg1_data.nrows):
    try:
        if type(idvg1_data.row(i)[0].value) is float:
            Vgs.append(idvg1_data.row(i)[1].value)
            Ids.append(idvg1_data.row(i)[3].value)
    except:
        continue

slope, intercept, r_value, p_value, std_err = stats.linregress(Vgs[130:150],Ids[130:150])
xintercept = -intercept / slope
#print slope, intercept, xintercept
xplot=linspace(-16.0,5.0,100)
yplot = slope * xplot + intercept
Vt = xintercept - Vds / 2.0
Cox = slope / (Vds*Mu*W/L)
Tox = 4.3*8.85E-14 / Cox #cm
Tox *= 1.0E4 #microns

figure()
ax1=axes([0.2,0.1,0.6,0.6])
ax1.set_title("STA3800C Id-Vg")
ax1.plot(Vgs, array(Ids)*1000.0, color = 'green', lw = 2)
ax1.plot(xplot, yplot*1000.0, color='blue', lw = 2, ls = '--')
ax1.set_xlabel("Vgs (volts)")
ax1.set_ylabel("Ids(mA)")
ax1.set_ylim(0,.6)

ax1.text(-18,0.5,"Vt = %.2f V"%Vt, fontsize=12)
ax1.text(-18,0.45,"Vds = %.1f V, W = %.1f um, L = %.1f um"%(Vds,W,L), fontsize=12)
#text(-18,004,"Tox = %.3f microns (?)"%Tox, fontsize=12)
savefig("IdVg1_8Mar17.pdf")

In [ ]:
# ID-VD 2
W= 28.0
L = 5.0
Mu = 1000.0
Vbs = 0.0
Vgs_start = -16.0
Vgs_step = 4.0
Vgs_num_steps = 8
Vds_num_steps = 251
Vds = zeros(Vds_num_steps)
Vgs = linspace(Vgs_start, Vgs_start + Vgs_step * (Vgs_num_steps - 1), Vgs_num_steps)
Ids = zeros([Vgs_num_steps, Vds_num_steps])
for i in range(idvd2_data.nrows):
    try:
        if type(idvd2_data.row(i)[0].value) is float:
            vds_index = (int(idvd2_data.row(i)[0].value) - 1) % Vds_num_steps
            vgs_index = int(round((idvd2_data.row(i)[1].value - Vgs_start) / Vgs_step))
            if vgs_index == 0:
                Vds[vds_index] = idvd2_data.row(i)[2].value
            Ids[vgs_index, vds_index] = idvd2_data.row(i)[3].value
    except:
        continue

fig = figure()
ax1=axes([0.2,0.1,0.6,0.6])
ax1.set_title("STA3800C Id-Vd")
for i in range(Vgs_num_steps-1, 1, -1):
    ax1.plot(Vds, Ids[i,:]*1000.0, label = 'Vgs = %.1f V'%Vgs[i], lw = 2)

ax1.set_xlabel("Vds (volts)")
ax1.set_ylabel("Ids(mA)")
ax1.set_ylim(0,8.0)
ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize = 6)
savefig("IdVd1_8Mar17.pdf")

In [ ]:
# ID-VG 3
W= 28.0
L = 5.0
Mu = 1000.0
Vds = 5.0
Vbs_start = 0.0
Vbs_step = -8.0
Vbs_num_steps = 2
Vgs_num_steps = 201
Vgs = zeros(Vgs_num_steps)
Vbs = linspace(Vbs_start, Vbs_start + Vbs_step * (Vbs_num_steps - 1), Vbs_num_steps)
Ids = zeros([Vbs_num_steps, Vgs_num_steps])
for i in range(idvg3_data.nrows):
    try:
        if type(idvg3_data.row(i)[0].value) is float:
            vgs_index = (int(idvg3_data.row(i)[0].value) - 1) % Vgs_num_steps
            vbs_index = int(round((idvg3_data.row(i)[2].value - Vbs_start) / Vbs_step))
            if vbs_index == 0:
                Vgs[vgs_index] = idvg3_data.row(i)[1].value
            Ids[vbs_index, vgs_index] = idvg3_data.row(i)[3].value
    except:
        continue

slope1, intercept1, r_value, p_value, std_err = stats.linregress(Vgs[130:150],Ids[0,130:150])
xintercept1 = -intercept1 / slope1
slope2, intercept2, r_value, p_value, std_err = stats.linregress(Vgs[130:150],Ids[1,130:150])
xintercept2 = -intercept2 / slope2
delta_vt = xintercept1 - xintercept2
gamma = delta_vt / sqrt(Vbs[0] - Vbs[1])
Tox = 0.1E-4 #cm
epsilon_ox = 8.85E-14 * 4.3
epsilon_si = 8.85E-14 * 11.7
qe = 1.6E-19
Nsub = gamma**2 * epsilon_ox**2 / (Tox**2*2.0*qe*epsilon_si)

fig = figure()
ax1=axes([0.2,0.1,0.6,0.6])
ax1.set_title("STA3800C Id-Vg")
for i in range(Vbs_num_steps-1, -1, -1):
    ax1.plot(Vgs, Ids[i,:]*1000.0, label = 'Vbs = %.1f V'%Vbs[i], lw = 2)

ax1.set_xlabel("Vgs (volts)")
ax1.set_ylabel("Ids(mA)")
ax1.set_ylim(0,8)
text(-18,7,"Vds = %.1f V, W = %.1f um, L = %.1f um"%(Vds,W,L), fontsize=12)
text(-18,6,"Delta-Vt = %.3f V"%delta_vt, fontsize=12)
text(-18,5,"Gamma = %.3f $V^{1/2}$"%gamma, fontsize=12)
text(-18,4,"Nsub = %.2g $cm^{-3}$"%Nsub, fontsize=12)

ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize = 6)
savefig("IdVg3_8Mar17.pdf")

In [ ]:
# C-V
MaxC1 = 0.0
MaxC2 = 0.0
Vgs1 = zeros([51])
C1 = zeros([51])
Vgs2 = zeros([11])
C2 = zeros([11])
for i in range(cv_data.nrows):
    try:
        if type(cv_data.row(i)[0].value) is float:
            index = int(round(cv_data.row(i)[0].value))
            if i < 52:
                Vgs1[index] = cv_data.row(i)[1].value
                C1[index] = cv_data.row(i)[2].value
                if C1[index] > MaxC1:
                    MaxC1 = C1[index]
            else:
                Vgs2[index] = cv_data.row(i)[1].value
                C2[index] = cv_data.row(i)[2].value
                if C2[index] > MaxC2:
                    MaxC2 = C2[index]

    except:
        continue
epsilon_ox = 8.85E-14 * 4.3
area = 3.33E-4 * 4072 * 4000 * 10.0E-4
Tox1 = epsilon_ox * area / (MaxC1 * 1.0E-9)
Tox2 = epsilon_ox * area / (MaxC2 * 1.0E-9)
print "MaxC1 = %f, Tox1 = %.4g"%(MaxC1,Tox1)
print "MaxC2 = %f, Tox2 = %.4g"%(MaxC2,Tox2)

fig = figure()
ax1=axes([0.2,0.1,0.6,0.6])
ax1.set_title("STA3800C C-V")
ax1.plot(Vgs1, C1, label = 'Light Off', lw = 2)
ax1.plot(Vgs2, C2, label = 'Light On', lw = 2)

ax1.set_xlabel("Vgs (volts)")
ax1.set_ylabel("C(nF)")
ax1.set_ylim(0,200)
text(-28,20,"Tox = %.3f - %.3f um"%(Tox2 * 1.0E4, Tox1 * 1.0E4), fontsize=12)
#text(-18,6,"Delta-Vt = %.3f V"%delta_vt, fontsize=12)
#text(-18,5,"Gamma = %.3f $V^{1/2}$"%gamma, fontsize=12)
#text(-18,4,"Nsub = %.2g $cm^{-3}$"%Nsub, fontsize=12)

ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize = 6)
savefig("CV_8Mar17.pdf")


In [ ]:
clear()